## KRWordRank: method for word / keyword extraction

KRWordRank는 Kim et al.(2014)^[1]의 논문을 바탕으로 한 비지도학습기반 단어 추출 기법으로, 데이터기반으로 주요단어 (키워드)를 추출하는 알고리즘이다. 하나의 도메인에 대한 문서들을 바탕으로 명사/형용사/동사/부사 (L set) 중에서 빈도수가 높거나, 주요 단어들과 함께 등장하는 단어를 키워드로 추출한다. KRWordRank는 이름에서 나타나는바와 같이 단어 후보 (subtokens)을 이용하여 word-graph를 생성한 뒤, PageRank의 랭킹 학습 방식을 이용하여 word-graph의 hub subtokens을 추출한다. 

KRWordRank는 다음의 가정을 기반으로 단어를 추출한다. **단어 주변에는 단어가 등장하며, 올바른 단어는 주위의 많은 단어들과 연결되어 있다. 그렇기 때문에 단어는 주위 단어들에 의하여 단어 점수가 보강(reinforced)된다.**


![kr_wordrank_structure](figs/kr_wordrank_fig1.png)


한국어는 의미를 지니는 단어 집합과 문법 기능을 하는 복합형태소 집합으로 나뉘어지며, [문법/명사] + [을/조사]와 같이 어절의 왼쪽에 의미를 지니는 단어인 명사/형용사/동사가 위치한다. 부사는 그 자체로 한 어절을 이룬다. 그렇기 때문에 KRWordRank는 의미있는 단어로서 어절 자체나 어절의 왼쪽에 등장하는 L set을 추출한다. 또한 한국어는 한 글자에 지나치게 많은 의미가 담겨져 있어 해석이 모호하기 때문에 1음절 단어는 추출되는 단어에서 제외한다. 실제로 subtokens으로 이뤄진 word-graph에서 1음절 단어들은 매우 높은 랭킹을 지닌다. KRWordRank는 아래 그림과 같이 subtokens을 어절의 위치에 따라 L/R tags를 부여하여 word-graph를 만든 뒤, 랭킹을 계산한다. 

![kr_wordrank_structure](figs/kr_wordrank_fig2.png)

논문에서 기술되지 않은 후처리(post-processing)가 추가되었다. 영화리뷰의 경우, '영화', '영화가', '영화를' 와 같이 "단어 + R set"이 함께 키워드로 추출된다. 이는 KRWordRank가 주요 L set 혹은 어절을 추출하기 때문이며, '영화', '영화가' 주변 모두 올바른 단어가 위치하기 때문이다. 그렇기 때문에 '영화'라는 단어가 '영화가', '영화를' 등보다 높은 랭킹을 지녔다면, '영화' + R set는 L + R 복합어라 판단하여 제외하였다. 

        keywords = self._select_keywords(lset, rset)

두번째 후처리로, '영화', '음악', '영화음악'이 키워드로 추출되었고, '영화', '음악'이 모두 '영화음악'보다 랭킹이 높을 경우, '영화음악'은 합성어로 판단하여 이를 제거하였다. 

        keywords = self._filter_compounds(keywords)

마지막 후처리로, '스토리'가 상위 랭킹이 될 경우, 한 글자가 랭킹이 높아서 '스토' 역시 키워드로 추출될 수 있다. '스토리'가 상위 랭킹이 된다면 '스토'와 같은 substring은 키워드에서 제거하였다. 

        keywords = self._filter_subtokens(keywords)

사용법은 아래의 예제 코드와 같다. 

[1] Kim, H. J., Cho, S., & Kang, P. (2014). KR-WordRank: An Unsupervised Korean Word Extraction Method Based on WordRank. Journal of Korean Institute of Industrial Engineers, 40(1), 18-33.

In [1]:
def get_texts_scores(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f]
        docs = [doc for doc in docs if len(doc) == 2]
        
        if not docs:
            return [], []
        
        texts, scores = zip(*docs)
        return list(texts), list(scores)

# La La Land
fname = '../data/134963.txt'
texts, scores = get_texts_scores(fname)

In [2]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize

단어 추출에 영어/숫자를 포함할 예정이라면 normalize함수를 이용하여 텍스트를 normalize할 것

In [3]:
with open('../data/134963_norm.txt', 'w', encoding='utf-8') as f:
    for text, score in zip(texts, scores):
        text = normalize(text, english=True, number=True)
        f.write('%s\t%s\n' % (text, str(score)))

In [4]:
# La La Land
fname = '../data/134963_norm.txt'
texts, scores = get_texts_scores(fname)

In [5]:
min_count = 5   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count, max_length)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
verbose = True
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter, verbose)

scan vocabs ... 
num vocabs = 15097
done


위와 같이 vocabulary를 미리 설정하거나 decaying factor를 단어별로 다르게 (bias) 할당할 수 있으며, 모든 단어의 랭킹의 총 합은 vocabulary size와 같음. 즉 default decaying factor는 1.0

In [6]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))

      영화:	229.7889
     관람객:	112.3404
      너무:	78.4055
      음악:	37.6247
      정말:	37.2504
     마지막:	34.9952
      최고:	22.4425
      사랑:	21.1355
     뮤지컬:	20.7357
      꿈을:	19.5282
     여운이:	19.4032
      보고:	19.4005
      아름:	18.6495
      진짜:	18.5599
      영상:	18.1099
      좋았:	17.8625
      노래:	16.9019
     스토리:	16.2600
      좋은:	15.4661
      그냥:	15.2136
      현실:	15.0772
      생각:	14.6264
      인생:	14.2642
      좋고:	13.9971
      지루:	13.8732
      다시:	13.7812
      감동:	13.4817
      느낌:	12.3127
      ㅠㅠ:	12.1447
      좋아:	11.9586


세 가지 영화의 키워드를 비교해보겠습니다. '라라랜드 (134963.txt)', '신세계 (91031.txt)', '엑스맨 (99714.txt)'에 대하여 동일한 방식으로 normalize를 한 뒤, 상위 100개의 키워드들을 비교해보겠습니다.

In [7]:
fnames = ['../data/91031.txt', '../data/99714.txt']
for fname in fnames:
    texts, scores = get_texts_scores(fname)
    with open(fname.replace('.txt', '_norm.txt'), 'w', encoding='utf-8') as f:
        for text, score in zip(texts, scores):
            text = normalize(text, english=True, number=True)
            f.write('%s\t%s\n' % (text, str(score)))

In [8]:
%%time
top_keywords = []
fnames = ['../data/134963_norm.txt', '../data/91031_norm.txt', '../data/99714_norm.txt']
for fname in fnames:
    texts, scores = get_texts_scores(fname)
    wordrank_extractor = KRWordRank(min_count, max_length)
    keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter, verbose=False)
    top_keywords.append(sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:100])

done
done
done
CPU times: user 10.4 s, sys: 32 ms, total: 10.5 s
Wall time: 10.5 s


In [9]:
movie_names = ['라라랜드', '신세계', '엑스맨']
for k in range(100):
    message = '  --  '.join(['%8s (%.3f)' % (top_keywords[i][k][0], top_keywords[i][k][1]) for i in range(3)])
    print(message)
    

      영화 (229.789)  --        영화 (145.228)  --       엑스맨 (106.492)
     관람객 (112.340)  --       황정민 (98.533)  --        영화 (70.255)
      너무 (78.405)  --        연기 (89.466)  --       관람객 (68.034)
      음악 (37.625)  --        정말 (74.272)  --       시리즈 (43.284)
      정말 (37.250)  --        진짜 (64.168)  --        진짜 (39.594)
     마지막 (34.995)  --        최고 (54.186)  --        정말 (38.611)
      최고 (22.443)  --        너무 (51.835)  --        너무 (38.027)
      사랑 (21.136)  --       이정재 (46.256)  --        재밌 (30.850)
     뮤지컬 (20.736)  --       무간도 (36.069)  --        최고 (29.771)
      꿈을 (19.528)  --       배우들 (33.970)  --        재미 (27.698)
     여운이 (19.403)  --        재밌 (28.617)  --       스토리 (23.497)
      보고 (19.401)  --       스토리 (26.657)  --        기대 (23.295)
      아름 (18.650)  --        한국 (26.173)  --        역시 (20.009)
      진짜 (18.560)  --       신세계 (24.254)  --        보고 (19.485)
      영상 (18.110)  --        대박 (23.215)  --        액션 (17.090)
      좋았 (17.862)  --       최민식 (19.

셋 모두 영화이기 때문에 공통된 키워드가 많습니다. top 100에서 중복되는 키워드들을 제거하고 차이가 있는 키워드만 추출해서 살펴보겠습니다. 

In [10]:
keyword_counter = {}
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    for word in words:
        keyword_counter[word] = keyword_counter.get(word, 0) + 1

common_keywords = {word for word, count in keyword_counter.items() if count == 3}
len(common_keywords)

43

세 영화 모두에 등장하는 키워드는 총 43개가 있으며, '스토리', '많이', '진짜' 같은 단어들입니다. 이런 단어를 제외한 selected_top_keywords 리스트를 만든 다음 출력을 해보겠습니다. 

In [11]:
str(common_keywords)

"{'지루', '다시', '하나', '조금', '아니', '기대', '스토리', '보면', 'ㅎㅎ', '좋았', '너무', '재미', '영화', '보는', '평점', '별로', '있는', '모두', '추천', '이렇게', '그냥', '그리고', '좋아', '하지만', '없다', '내가', '한번', '이런', '보고', '생각', '봤는데', '느낌', '진짜', '정말', '처음', 'ㅠㅠ', '마지막', '최고', '재밌', '많이', '사람', '가장', '내용'}"

In [12]:
selected_top_keywords = []
for keywords in top_keywords:
    selected_keywords = []
    for word, r in keywords:
        if word in common_keywords:
            continue
        selected_keywords.append((word, r))
    selected_top_keywords.append(selected_keywords)

In [13]:
def get_from_list(l, i, default=('', 0)):
    if len(l) <= i:
        return default
    else:
        return l[i]

라라랜드는 [음악, 사랑, 뮤지컬, 꿈]과 같은 단어들이 나오며, 신세계에서는 [황정민, 이정재, 최민식]과 같은 배우들의 이름과, 홍콩영화 무간도와 주제가 비슷하기에 '무간도'라는 단어, 그리고 ['조폭', '느와르', 잔인'] 같은 영화 분위기와 관련된 내용들이 나옵니다. 또한 '반전'이란 단어에서 반전이 있는 영화라는 것도 알 수 있겠네요. 그에 비하여 엑스맨에서는 캐릭터 이름인 ['울버린', '퀵실버'] 같은 단어들도 나옵니다. ['꿀잼', '마블']과 같은 단어로부터 마블 코믹스의 오락 영화라는 것도 알 수 있습니다. 

In [15]:
for k in range(100 - len(common_keywords) ):
    message = '  --  '.join(['%8s (%.3f)' % get_from_list(selected_top_keywords[i], k) for i in range(3)])
    print(message)
    

     관람객 (112.340)  --       황정민 (98.533)  --       엑스맨 (106.492)
      음악 (37.625)  --        연기 (89.466)  --       관람객 (68.034)
      사랑 (21.136)  --       이정재 (46.256)  --       시리즈 (43.284)
     뮤지컬 (20.736)  --       무간도 (36.069)  --        역시 (20.009)
      꿈을 (19.528)  --       배우들 (33.970)  --        액션 (17.090)
     여운이 (19.403)  --        한국 (26.173)  --        ㅋㅋ (15.580)
      아름 (18.650)  --       신세계 (24.254)  --        싱어 (15.251)
      영상 (18.110)  --        대박 (23.215)  --       퍼스트 (14.674)
      노래 (16.902)  --       최민식 (19.561)  --        진심 (14.637)
      좋은 (15.466)  --       느와르 (19.515)  --        명작 (12.979)
      현실 (15.077)  --        ㅋㅋ (19.317)  --        완전 (11.447)
      인생 (14.264)  --        완전 (16.529)  --        제일 (11.105)
      좋고 (13.997)  --        잔인 (13.826)  --        이건 (9.705)
      감동 (13.482)  --        역시 (13.320)  --       울버린 (9.203)
      계속 (11.508)  --        조폭 (12.819)  --        이번 (9.149)
      연기 (11.479)  --        특히 (12.722) 